In [10]:
import sys  
sys.path.insert(1, '../../../')

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from classifiers.nn.siamese.siamese_model import SiameseNetwork
from datasets.siamese_dataset import SiameseNetworkDataset
from losses.contrastive_loss import ContrastiveLoss
from torch import linalg as LA

In [15]:
# Load the training dataset
folder_dataset = datasets.ImageFolder(root="./data/faces/training/")

# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((100, 100)),
                                     transforms.ToTensor()
                                     ])

# Initialize the network
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transformation)

# Create a simple dataloader just for simple visualization
vis_dataloader = DataLoader(siamese_dataset,
                            shuffle=True,
                            num_workers=2,
                            batch_size=8)

# Extract one batch
example_batch = next(iter(vis_dataloader))

# Example batch is a list containing 2x8 images, indexes 0 and 1, an also the label
# If the label is 1, it means that it is not the same person, label is 0, same person in both images
concatenated = torch.cat((example_batch[0], example_batch[1]), 0)

# Load the training dataset
train_dataloader = DataLoader(siamese_dataset,
                              shuffle=True,
                              num_workers=8,
                              batch_size=64)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SiameseNetwork().to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

counter = []
loss_history = []
iteration_number = 0

In [17]:
# Iterate throught the epochs
for epoch in range(100):

    # Iterate over batches
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):

        # Send the images and labels to CUDA
        img0, img1, label = img0.to(device), img1.to(device), label.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0:
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

Epoch number 0
 Current loss 2.246217727661133

Epoch number 1
 Current loss 0.9660587906837463

Epoch number 2
 Current loss 1.475167989730835

Epoch number 3
 Current loss 0.9903945922851562

Epoch number 4
 Current loss 0.7071725726127625

Epoch number 5
 Current loss 0.608333170413971

Epoch number 6
 Current loss 0.7438515424728394

Epoch number 7
 Current loss 0.6570637822151184

Epoch number 8
 Current loss 0.6957849860191345

Epoch number 9
 Current loss 0.4180053472518921

Epoch number 10
 Current loss 0.8747988939285278

Epoch number 11
 Current loss 0.5132339596748352

Epoch number 12
 Current loss 0.545182466506958

Epoch number 13
 Current loss 0.7953558564186096

Epoch number 14
 Current loss 0.4102385342121124

Epoch number 15
 Current loss 0.36216849088668823

Epoch number 16
 Current loss 0.3699289560317993

Epoch number 17
 Current loss 0.38522979617118835

Epoch number 18
 Current loss 0.23519006371498108

Epoch number 19
 Current loss 0.5001189708709717

Epoch numbe

In [39]:
# Locate the test dataset and load it into the SiameseNetworkDataset
folder_dataset_test = datasets.ImageFolder(root="./data/faces/testing/")
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transformation)
test_dataloader = DataLoader(siamese_dataset, num_workers=2, batch_size=1, shuffle=True)

# Grab one image that we are going to test
dataiter = iter(test_dataloader)
x0, _, _ = next(dataiter)

In [45]:
for i in range(5):
    # Iterate over 5 images and test them with the first image (x0)
    _, x1, label2 = next(dataiter)

    # Concatenate the two images together
    concatenated = torch.cat((x0, x1), 0)

    output1, output2 = net(x0.to(device), x1.to(device))
    euclidean_distance = F.pairwise_distance(output1, output2)
    euclidean_distance2 = F.pairwise_distance(LA.norm(output1), LA.norm(output2))
    print(f'Dissimilarity: {euclidean_distance.item():.2f} {1 / (euclidean_distance.item() + 1):.2f} {euclidean_distance2.item():.2f}')

Dissimilarity: 4.53 0.18 2.29
Dissimilarity: 0.87 0.53 0.23
Dissimilarity: 5.42 0.16 0.41
Dissimilarity: 1.66 0.38 0.97
Dissimilarity: 5.42 0.16 0.41
